In [ ]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import numpy as np




In [ ]:
pwd

In [ ]:
# set directory to the plot_folder folder
os.chdir('/net/home/dmederer/Transferability_efficient/plot_folder')

In [ ]:
# Load the data from CSV into a DataFrame
data_predictions_start = pd.read_csv('Predictions_combined_rtm_only.csv') 
data_observations_start = pd.read_csv('Observations_combined_rtm_only.csv') 

In [ ]:
# Remove the suffix from the column names in data_predictions
data_predictions_start.columns = data_predictions_start.columns.str.replace(' Predictions', '')
data_predictions_start



In [ ]:
# Now use the same concept as in the cell before to loop through all the variables
# and print the slope and intercept for each variable
# Print it with proper captions
for column in data_predictions_start.columns:
    data_observations = data_observations_start
    data_predictions = data_predictions_start
    # remove all rows in column 'LMA (g/m²)' with NaN values from data_observations
    data_observations = data_observations.dropna(subset=[column])
    # using the index remove all rows from data_predictions that are not in data_observations
    data_predictions = data_predictions[data_predictions.index.isin(data_observations.index)]
    X = data_predictions[column].values.reshape(-1, 1)
    y = data_observations[column].values.reshape(-1, 1)
    model = LinearRegression()
    model.fit(X, y)

    # Obtain residuals
    y_pred = model.predict(X)
    residuals = y - y_pred

    # Calculate residual standard error (RSE) or root mean squared error (RMSE)
    n = len(y)
    p = X.shape[1] if isinstance(X, np.ndarray) else X.shape[1] + 1
    rse = np.sqrt(np.sum(residuals**2) / (n - p))

    # save the print statements to a text file
    with open('Slope_intercept_rtm_only.txt', 'a') as f:
        print('Slope: ', model.coef_[0][0], file=f)
        print('Intercept: ', model.intercept_[0], file=f)
        print('RSE: ', rse, file=f)
        print('Variable: ', column, file=f)
        print(' ', file=f)

    





In [ ]:
# SAME THING FOR REAL DATA

# Load the data from CSV into a DataFrame
data_predictions_start = pd.read_csv('Predictions_combined_real_only.csv') 
data_observations_start = pd.read_csv('Observations_combined_real_only.csv') 

# Remove the suffix from the column names in data_predictions
data_predictions_start.columns = data_predictions_start.columns.str.replace(' Predictions', '')
data_predictions_start

# Now use the same concept as in the cell before to loop through all the variables
# and print the slope and intercept for each variable
# Print it with proper captions
for column in data_predictions_start.columns:
    data_observations = data_observations_start
    data_predictions = data_predictions_start
    # remove all rows in column 'LMA (g/m²)' with NaN values from data_observations
    data_observations = data_observations.dropna(subset=[column])
    # using the index remove all rows from data_predictions that are not in data_observations
    data_predictions = data_predictions[data_predictions.index.isin(data_observations.index)]
    X = data_predictions[column].values.reshape(-1, 1)
    y = data_observations[column].values.reshape(-1, 1)
    model = LinearRegression()
    model.fit(X, y)
    # Obtain residuals
    y_pred = model.predict(X)
    residuals = y - y_pred

    # Calculate residual standard error (RSE) or root mean squared error (RMSE)
    n = len(y)
    p = X.shape[1] if isinstance(X, np.ndarray) else X.shape[1] + 1
    rse = np.sqrt(np.sum(residuals**2) / (n - p))

    # save the print statements to a text file
    with open('Slope_intercept_real_only.txt', 'a') as f:
        print('Slope: ', model.coef_[0][0], file=f)
        print('Intercept: ', model.intercept_[0], file=f)
        print('RSE: ', rse, file=f)
        print('Variable: ', column, file=f)
        print(' ', file=f)

In [ ]:
# Plot predicted vs. observed values
plt.figure(figsize=(8, 6))
plt.scatter(X, y, color='blue', label='Observed vs. Predicted')
plt.plot(X, model.predict(X), color='red', label='Linear Regression')
plt.xlabel('Predicted Values')
plt.ylabel('Observed Values')
# make sure that x and y axis have the same scale, using the maximum value for y
plt.xlim(0, max(y))
plt.ylim(0, max(y))
#Include the slope and intercept in the down right corner of the plot
plt.text(0.1, 0.9, 'Slope: ' + str(round(model.coef_[0][0], 2)), transform=plt.gca().transAxes)
plt.text(0.1, 0.8, 'Intercept: ' + str(round(model.intercept_[0], 2)), transform=plt.gca().transAxes)
plt.title('Assessing Bias with Linear Regression')
plt.legend()
plt.show()

In [ ]:
# TEST RESIDUALS
from scipy.stats import wilcoxon

# Load the data from CSV into a DataFrame
data_predictions_rtm = pd.read_csv('Predictions_combined_rtm_only.csv')
data_observations_rtm = pd.read_csv('Observations_combined_rtm_only.csv')

data_predictions_real_only = pd.read_csv('Predictions_combined_real_only.csv')
data_observations_real_only = pd.read_csv('Observations_combined_real_only.csv')

# Remove the suffix from the column names in data_predictions
data_predictions_rtm.columns = data_predictions_rtm.columns.str.replace(' Predictions', '')
data_predictions_real_only.columns = data_predictions_real_only.columns.str.replace(' Predictions', '')


for column in data_predictions_rtm.columns:
    data_observations_rtm_active = data_observations_rtm
    data_predictions_rtm_active = data_predictions_rtm

    data_observations_real_active = data_observations_real_only
    data_predictions_real_active = data_predictions_real_only

    # remove all rows in column with NaN values from data_observations
    data_observations_rtm_active = data_observations_rtm_active.dropna(subset=[column])
    data_observations_real_active = data_observations_real_active.dropna(subset=[column])
    # using the index remove all rows from data_predictions that are not in data_observations
    data_predictions_rtm_active = data_predictions_rtm_active[data_predictions_rtm_active.index.isin(data_observations_rtm_active.index)]
    data_predictions_real_active = data_predictions_real_active[data_predictions_real_active.index.isin(data_observations_real_active.index)]

    # Calculate residuals
    residuals_rtm = data_observations_rtm_active[column] - data_predictions_rtm_active[column]
    residuals_real = data_observations_real_active[column] - data_predictions_real_active[column]

    # Perform one-sided Wilcoxon signed-rank test
    test_statistic, p_value = wilcoxon(residuals_rtm, residuals_real, alternative='less')

    # Save the results to a text file
    with open('Wilcoxon_test_less_rtm_only.txt', 'a') as f:
        print('Variable: ', column, file=f)
        print(f"Test Statistic: {test_statistic}", file=f)
        print(f"P-value: {p_value}", file=f)
        print(' ', file=f)



In [ ]:
residuals_rtm

In [ ]:
# Now load the two text files with the slope and intercept values
# and save them to a DataFrame

import pandas as pd

# Read the text file
with open('Slope_intercept_rtm.txt', 'r') as file:
    lines = file.readlines()

# Initialize lists to store parsed values
slope = []
intercept = []
variable = []
rse = []

# Parse the content and extract values
for i in range(0, len(lines), 5):
    slope_val = float(lines[i].split(':')[1].strip())
    intercept_val = float(lines[i + 1].split(':')[1].strip())
    rse_val = float(lines[i + 2].split(':')[1].strip())
    variable_val = lines[i + 3].split(':')[1].strip()
    
    slope.append(slope_val)
    intercept.append(intercept_val)
    variable.append(variable_val)
    rse.append(rse_val)

# Create a DataFrame
data = {
    'Slope': slope,
    'Intercept': intercept,
    'RSE': rse,
    'Variable': variable
}

df_rtm = pd.DataFrame(data)

print(df_rtm)


# Read the text file
with open('Slope_intercept_real_only.txt', 'r') as file:
    lines = file.readlines()

# Initialize lists to store parsed values
slope = []
intercept = []
variable = []
rse = []

# Parse the content and extract values
for i in range(0, len(lines), 5):
    slope_val = float(lines[i].split(':')[1].strip())
    intercept_val = float(lines[i + 1].split(':')[1].strip())
    rse_val = float(lines[i + 2].split(':')[1].strip())
    variable_val = lines[i + 3].split(':')[1].strip()
    
    slope.append(slope_val)
    intercept.append(intercept_val)
    variable.append(variable_val)
    rse.append(rse_val)

# Create a DataFrame
data = {
    'Slope': slope,
    'Intercept': intercept,
    'RSE': rse,
    'Variable': variable
}

df_real_only = pd.DataFrame(data)

print(df_real_only)


In [ ]:
# For each variable, calculate the difference between the slopes of the two models and divide it
# by the root of the difference of the standard errors of the two slopes


import numpy as np

# Calculate the effect size for each variable
result_values = []

variables = df_rtm['Variable'].unique()

for var in variables:
    slope_values_1 = df_rtm[df_rtm['Variable'] == var]['Slope']
    slope_values_2 = df_real_only[df_real_only['Variable'] == var]['Slope']

    rse_values_1 = df_rtm[df_rtm['Variable'] == var]['RSE']
    rse_values_2 = df_real_only[df_real_only['Variable'] == var]['RSE']
    
    # Calculate the difference between the slopes
    slope_difference = slope_values_1.mean() - slope_values_2.mean()
    
    # Calculate the root of the sum of the standard errors
    se_root_sum = np.sqrt(rse_values_1.mean()**2 + rse_values_2.mean()**2)
    
    # Calculate the effect size
    test_val = slope_difference / se_root_sum

    result_values.append(var)
    result_values.append(test_val)

In [ ]:
result_values

In [ ]:
import pandas as pd
from scipy.stats import mannwhitneyu

# Perform Mann-Whitney U test for each variable
variables = df_rtm['Variable'].unique()

for var in variables:
    slope_values_1 = df_rtm[df_rtm['Variable'] == var]['Slope']
    slope_values_2 = df_real_only[df_real_only['Variable'] == var]['Slope']
    
    # Perform the Mann-Whitney U test
    statistic, p_value = mannwhitneyu(slope_values_1, slope_values_2)
    
    print(f"Variable: {var}")
    print(f"Mann-Whitney U Statistic: {statistic}")
    print(f"P-value: {p_value}")
    print(f"Significant Difference: {p_value < 0.05}")  # Assuming alpha = 0.05
    print("-----------------------------")

In [ ]:
## Additional step: get absolute values of residuals and plot their distribution 

# Load the data from CSV into a DataFrame
data_predictions_rtm = pd.read_csv('Predictions_combined_low_var.csv')
data_observations_rtm = pd.read_csv('Observations_combined_low_var.csv')

data_predictions_real_only = pd.read_csv('Predictions_combined_real_only.csv')
data_observations_real_only = pd.read_csv('Observations_combined_real_only.csv')

# Remove the suffix from the column names in data_predictions
data_predictions_rtm.columns = data_predictions_rtm.columns.str.replace(' Predictions', '')
data_predictions_real_only.columns = data_predictions_real_only.columns.str.replace(' Predictions', '')


for column in data_predictions_rtm.columns:
    data_observations_rtm_active = data_observations_rtm
    data_predictions_rtm_active = data_predictions_rtm

    data_observations_real_active = data_observations_real_only
    data_predictions_real_active = data_predictions_real_only

    # remove all rows in column with NaN values from data_observations
    data_observations_rtm_active = data_observations_rtm_active.dropna(subset=[column])
    data_observations_real_active = data_observations_real_active.dropna(subset=[column])
    # using the index remove all rows from data_predictions that are not in data_observations
    data_predictions_rtm_active = data_predictions_rtm_active[data_predictions_rtm_active.index.isin(data_observations_rtm_active.index)]
    data_predictions_real_active = data_predictions_real_active[data_predictions_real_active.index.isin(data_observations_real_active.index)]

    # Calculate absolute values of residuals
    residuals_rtm = np.abs(data_observations_rtm_active[column] - data_predictions_rtm_active[column])
    residuals_real = np.abs(data_observations_real_active[column] - data_predictions_real_active[column])

In [ ]:
residuals_rtm

In [ ]:
residuals_real

In [ ]:
# Plot the distribution of residuals
import matplotlib.pyplot as plt
import seaborn as sns


plt.figure(figsize=(8, 6))
sns.distplot(residuals_rtm, label='RTM')
sns.distplot(residuals_real, label='Real')
plt.xlabel('Absolute Residuals')
plt.ylabel('Density')
plt.title('Distribution of Absolute Residuals')
plt.legend()
plt.show()

